In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [3]:
# loading in dataset
df = pd.read_csv("/Users/cartermain/Downloads/BankChurners.csv")

In [4]:
# getting familiar with dataframe
print(df.head(3))

   CLIENTNUM     Attrition_Flag  Customer_Age Gender  Dependent_count  \
0  768805383  Existing Customer            45      M                3   
1  818770008  Existing Customer            49      F                5   
2  713982108  Existing Customer            51      M                3   

  Education_Level Marital_Status Income_Category Card_Category  \
0     High School        Married     $60K - $80K          Blue   
1        Graduate         Single  Less than $40K          Blue   
2        Graduate        Married    $80K - $120K          Blue   

   Months_on_book  ...  Months_Inactive_12_mon  Contacts_Count_12_mon  \
0              39  ...                       1                      3   
1              44  ...                       1                      2   
2              36  ...                       1                      0   

   Credit_Limit  Total_Revolving_Bal  Avg_Open_To_Buy  Total_Amt_Chng_Q4_Q1  \
0       12691.0                  777          11914.0                 

We're going to use a logistic regression model to predict churn probability for cardholders but first we need to clean up our dataset

In [5]:
# counting null values within dataframe
print(df.isnull().sum())

CLIENTNUM                   0
Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
dtype: int64


In [6]:
# using interquartile range to turn outliers of columns with int and float dtypes into null values
for feature in df.columns:
    if df[feature].dtype == "object":
        continue
    else:
        for x in df[feature].sort_values(ascending = False)[0:100]:
            q75 = df[feature].describe()["75%"]
            q25 = df[feature].describe()["25%"]
            intr_qr = q75 - q25
            max = q75 + (1.5 * intr_qr)
            min = q25 - (1.5 * intr_qr)
            if x > max or x < min:
                df.loc[df[feature] == x, feature] = np.nan
        for x in df[feature][df[feature].notnull() == True].sort_values(ascending = False)[-100:]:
            if x > max or x < min:
                df.loc[df[feature] == x, feature] = np.nan

In [7]:
# counting null values after dropping outliers 
print(df.isnull().sum())

CLIENTNUM                     0
Attrition_Flag                0
Customer_Age                  2
Gender                        0
Dependent_count               0
Education_Level               0
Marital_Status                0
Income_Category               0
Card_Category                 0
Months_on_book              223
Total_Relationship_Count      0
Months_Inactive_12_mon      153
Contacts_Count_12_mon       629
Credit_Limit                508
Total_Revolving_Bal           0
Avg_Open_To_Buy             100
Total_Amt_Chng_Q4_Q1        148
Total_Trans_Amt             101
Total_Trans_Ct                2
Total_Ct_Chng_Q4_Q1         203
Avg_Utilization_Ratio         0
dtype: int64


In [8]:
# dropping null values our for loop identified as outliers
df.dropna(inplace = True)

We're going to need to turn all columns into an int or float dtype to work within our logistic regression model

In [9]:
# using label encoder to turn classification columns with "object" dtype into int 
# and collecting column names to drop when we create feature set
le = preprocessing.LabelEncoder()
object_columns = []
for column in df.columns:
    if df[column].dtype == "object":
        df[column + "_class"] = le.fit_transform(df[column])
        object_columns.append(column)

In [10]:
# collecting features and dropping columns we collected at last step with "object" dtype 
# as well as our target and client number
features = df.drop(columns = object_columns)
features = features.drop(columns = "Attrition_Flag_class")
features = features.drop(columns = "CLIENTNUM")

In [11]:
# scaling feature set for use in logistic regression model since features are on different sclaes
scaler = StandardScaler()
scaler.fit_transform(features)

array([[-0.29929769, -0.28936318,  0.00837525, ..., -0.62768276,
        -1.28941323, -0.1971452 ],
       [-0.5594046 ,  2.04872872, -0.66682905, ...,  2.07358903,
        -1.96567778, -0.1971452 ],
       [ 2.43182492, -1.06872715,  2.43911075, ..., -0.62768276,
        -1.28941323, -0.1971452 ],
       ...,
       [-0.29929769, -1.06872715,  0.00837525, ..., -0.62768276,
         0.73938041, -0.1971452 ],
       [-2.12004609, -0.28936318,  0.00837525, ...,  2.07358903,
        -1.28941323, -0.1971452 ],
       [-0.42935115, -0.28936318, -1.47707421, ..., -0.62768276,
         0.73938041,  5.24458467]])

In [12]:
# train test splitting with random state of 42 so we can accurately assess accuracy improvements later on
x_train, x_test, y_train, y_test = train_test_split(features, df["Attrition_Flag_class"], train_size = 0.7, random_state = 42)

In [13]:
# fitting and scoring the first iteration of our logistic regression model
lr = LogisticRegression(max_iter = 5000)
lr.fit(x_train, y_train)
print(lr.score(x_test, y_test))

0.884046381447421


A strong start. Let's run feature selection via RFE to see if we can match or improve our model's accuracy with fewer features

In [14]:
# running RFE through a for loop to determine which combination of features yield the highest accuracy
max_score = 0
best_x = 0
for x in range(1,len(features.columns) + 1):
    rfe = RFE(estimator = lr, n_features_to_select = x)
    rfe.fit(x_train, y_train)
    score = rfe.score(x_test, y_test)
    if score > max_score:
        max_score = score
        best_x = x
print(max_score, best_x)

0.8932427029188325 17


In [15]:
# training RFE outside of loop with best number of features so we can find the features it recommends keeping
rfe = RFE(estimator = lr, n_features_to_select = best_x)
rfe.fit(x_train, y_train)
print(rfe.support_)

[ True  True  True  True  True  True False  True False  True  True  True
  True  True  True  True  True  True  True]


In [16]:
# collecting the feature names we will end up keeping in our revised feature set
kept_features = []
x = 0
for kept in rfe.support_:
    if kept == True:
        kept_features.append(features.columns[x])
    x += 1

In [17]:
# creating new feature set with the 17 features RFE determined generated the highest accuracy
features_2 = df[kept_features]

In [18]:
# scaling our revised feature set for use in logistic regression
scaler_2 = StandardScaler()
scaler_2.fit_transform(features_2)

array([[-0.29929769, -0.28936318,  0.00837525, ..., -0.62768276,
        -1.28941323, -0.1971452 ],
       [-0.5594046 ,  2.04872872, -0.66682905, ...,  2.07358903,
        -1.96567778, -0.1971452 ],
       [ 2.43182492, -1.06872715,  2.43911075, ..., -0.62768276,
        -1.28941323, -0.1971452 ],
       ...,
       [-0.29929769, -1.06872715,  0.00837525, ..., -0.62768276,
         0.73938041, -0.1971452 ],
       [-2.12004609, -0.28936318,  0.00837525, ...,  2.07358903,
        -1.28941323, -0.1971452 ],
       [-0.42935115, -0.28936318, -1.47707421, ..., -0.62768276,
         0.73938041,  5.24458467]])

In [19]:
# train test splitting revised feature set with same random state split
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(features_2, df["Attrition_Flag_class"], train_size = 0.7, random_state = 42)

In [20]:
# retraining our logistic regression model with our split and scaled new feature set
lr.fit(x_train_2, y_train_2)
print(lr.score(x_test_2, y_test_2))

0.8932427029188325


Nice, we were able to improve the accuracy of our model and drop 2 features. Now let's use random search to check if there are any better-performing hyperparameters than the logistic regression defaults

In [21]:
# setting up dictionary of hyperparameters (solver and C values) to test via randomized search
params = {"solver": ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"], "C": range(0, 30, 5)}

In [22]:
# running our random search to indentify any hyperparameter combinations that outperform defaults
# some of these won't converge but 
randomsearch = RandomizedSearchCV(lr, params, n_iter = 20)
randomsearch.fit(x_train_2, y_train_2)

/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
35 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_m

RandomizedSearchCV(estimator=LogisticRegression(max_iter=5000), n_iter=20,
                   param_distributions={'C': range(0, 30, 5),
                                        'solver': ['lbfgs', 'liblinear',
                                                   'newton-cg',
                                                   'newton-cholesky', 'sag',
                                                   'saga']})

In [23]:
# finding best score from our random search test and corresponding hyperparameters
print(randomsearch.score(x_test_2, y_test_2))
print(randomsearch.best_params_)

0.8972411035585766
{'solver': 'newton-cg', 'C': 15}


We were able to find a slightly better score than our default values with newton_cg as the solver and a C of 10. Let's try a few different C values via gridsearchCV and compare the default solver with newton-cg

In [24]:
# setting up a gridsearchCV to test a few targeted hyperparams based on results of randomsearch
gridsearch_params = {"solver": ["newton-cg", "lbfgs"], "C": range(5, 20)}

In [25]:
# running gridsearchCV with the different C values identified in the previous step
gridsearch = GridSearchCV(lr, gridsearch_params)
gridsearch.fit(x_train_2, y_train_2)

/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:415: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/cartermain/opt/miniconda3/lib

GridSearchCV(estimator=LogisticRegression(max_iter=5000),
             param_grid={'C': range(5, 20), 'solver': ['newton-cg', 'lbfgs']})

In [26]:
# finding best score and corresponding params
print(gridsearch.score(x_test_2, y_test_2))
print(gridsearch.best_params_)

0.8972411035585766
{'C': 9, 'solver': 'newton-cg'}


Let's update our live model with the hyperparams that resulted in the most accurate model

In [27]:
# updating our logistic regression model based on best found C value and solver
lr = LogisticRegression(solver = gridsearch.best_params_["solver"], C = gridsearch.best_params_["C"], max_iter = 5000)
lr.fit(x_train_2, y_train_2)
print(lr.score(x_test_2, y_test_2))

0.8972411035585766


As a final step, let's test different train sizes for our split to find the train size that results in the highest accuracy score for our model. 

In [28]:
# now, let's find the best train test split that avoids overfitting while using same random state to ensure a 1:1 comparison
best_score = 0
best_split = 0
for x in np.linspace(0,1,21):
    if x == 0 or x == 1:
        continue
    else:
        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(features_2, df["Attrition_Flag_class"], train_size = x, random_state = 42)
        lr.fit(x_train_3, y_train_3)
        score = lr.score(x_test_3, y_test_3)
        if score > best_score:
            best_score = score
            best_split = x
print(best_score, best_split)

/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:415: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:415: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not conv

0.9054146675805346 0.30000000000000004


Picked up another incremental accuracy improvement. Let's update our train test split to train our final model using the split that resulted in the most accuracy

In [29]:
# splitting our dataset one final time with the best found train size
best_x_train, best_x_test, best_y_train, best_y_test = train_test_split(features_2, df["Attrition_Flag_class"], train_size = best_split, random_state = 42)

In [30]:
# training and scoring our model with the newly-created train test split
lr.fit(best_x_train, best_y_train)
print(lr.score(best_x_test, best_y_test))

0.9054146675805346


In [39]:
# finding coefficient of each feature in final version of our Logistic Regresison model
coef_df = pd.DataFrame(index = features_2.columns, columns = ["coef"], data = lr.coef_[0])
print(coef_df)

                              coef
Customer_Age              0.016091
Dependent_count          -0.122481
Months_on_book            0.006188
Total_Relationship_Count  0.460549
Months_Inactive_12_mon   -0.616461
Contacts_Count_12_mon    -0.299435
Total_Revolving_Bal       0.001018
Total_Amt_Chng_Q4_Q1      0.426151
Total_Trans_Amt          -0.000466
Total_Trans_Ct            0.125815
Total_Ct_Chng_Q4_Q1       3.108323
Avg_Utilization_Ratio    -0.156064
Gender_class              0.644884
Education_Level_class    -0.023345
Marital_Status_class     -0.313056
Income_Category_class    -0.052724
Card_Category_class       0.192352


We were able to get our model's accuracy over 90% with a few optimizations. Now, let's run a few sample customers through and find the probability that they will churn.

In [40]:
# loading in sample dataset
sample_df = pd.read_csv("/Users/cartermain/Downloads/Churn Sample Set.csv")

In [41]:
# getting familiar with dataset
print(sample_df.head(3))

   Customer_Age  Dependent_count  Months_on_book  Total_Relationship_Count  \
0            48                2              30                         4   
1            53                0              46                         2   
2            28                0              20                         0   

   Months_Inactive_12_mon  Contacts_Count_12_mon  Total_Revolving_Bal  \
0                       0                      1                  750   
1                       0                      3                  617   
2                       2                      5                   86   

   Total_Amt_Chng_Q4_Q1  Total_Trans_Amt  Total_Trans_Ct  Total_Ct_Chng_Q4_Q1  \
0                  0.78             2870              21                 0.33   
1                  0.81             8223              62                 0.76   
2                  0.25              609              30                 0.54   

   Avg_Utilization_Ratio  Gender_class  Education_Level_class  \
0   

In [42]:
print(lr.predict_proba(sample_df))

[[0.69944776 0.30055224]
 [0.13426406 0.86573594]
 [0.98426819 0.01573181]
 [0.7848634  0.2151366 ]
 [0.00147378 0.99852622]]


In [43]:
# adding model-predicted probability of churn to our sample dataframe as a new column
sample_df["churn_proba"] = lr.predict_proba(sample_df)[:,0]

In [44]:
# adding model-predicted churn class to our sample dataframe as new column based on churn probability
sample_df["churn_class"] = sample_df["churn_proba"].apply(lambda row: 0 if row >= 0.50 else 1)

In [45]:
# printing head to view results
print(sample_df.head())

   Customer_Age  Dependent_count  Months_on_book  Total_Relationship_Count  \
0            48                2              30                         4   
1            53                0              46                         2   
2            28                0              20                         0   
3            36                4              12                         3   
4            68                2              87                         5   

   Months_Inactive_12_mon  Contacts_Count_12_mon  Total_Revolving_Bal  \
0                       0                      1                  750   
1                       0                      3                  617   
2                       2                      5                   86   
3                       1                      0                    0   
4                       0                      2                    0   

   Total_Amt_Chng_Q4_Q1  Total_Trans_Amt  Total_Trans_Ct  Total_Ct_Chng_Q4_Q1  \
0          